<b>CS225 - Project 3</b>
<br/>
U-ASK is a unified indexing and query processing for kNN spatial-keyword queries supporting negative keyword predicates

In [1]:
class Point:
    def __init__(self, id, x, y, keywords):
        self.id = id
        self.x = x
        self.y = y
        self.keywords = keywords


In [2]:
class QuadTreeNode:
    def __init__(self, x0, y0, x1, y1, capacity=4, max_depth=10, depth=0):
        self.bounds = (x0, y0, x1, y1)
        self.capacity = capacity
        self.points = []
        self.divided = False
        self.children = []
        self.max_depth = max_depth
        self.depth = depth

    def insert(self, point):
        stack = [(self, point)]  # Stack of (node, point) pairs
        while stack:
            node, current_point = stack.pop()
            if not node.in_bounds(current_point):
                continue

            if len(node.points) < node.capacity or node.depth >= node.max_depth:
                node.points.append(current_point)
                continue

            if not node.divided:
                node.subdivide()

            for child in node.children:
                stack.append((child, current_point))

    def subdivide(self):
        x0, y0, x1, y1 = self.bounds
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2

        self.children = [
            QuadTreeNode(x0, y0, mid_x, mid_y, self.capacity, self.max_depth, self.depth + 1),
            QuadTreeNode(mid_x, y0, x1, mid_y, self.capacity, self.max_depth, self.depth + 1),
            QuadTreeNode(x0, mid_y, mid_x, y1, self.capacity, self.max_depth, self.depth + 1),
            QuadTreeNode(mid_x, mid_y, x1, y1, self.capacity, self.max_depth, self.depth + 1),
        ]
        self.divided = True




    def in_bounds(self, point):
        x0, y0, x1, y1 = self.bounds
        return x0 <= point.x <= x1 and y0 <= point.y <= y1

    def query_range(self, x0, y0, x1, y1):
        points_in_range = []
        if not self.intersects(x0, y0, x1, y1):
            return points_in_range

        for point in self.points:
            if x0 <= point.x <= x1 and y0 <= point.y <= y1:
                points_in_range.append(point)

        if self.divided:
            for child in self.children:
                points_in_range.extend(child.query_range(x0, y0, x1, y1))

        return points_in_range

    def intersects(self, x0, y0, x1, y1):
        qx0, qy0, qx1, qy1 = self.bounds
        return not (x1 < qx0 or x0 > qx1 or y1 < qy0 or y0 > qy1)


In [3]:
class InvertedIndex:
    def __init__(self):
        self.index = {}

    def insert(self, point):
        for keyword in point.keywords:
            if keyword not in self.index:
                self.index[keyword] = set()
            self.index[keyword].add(point.id)

    def filter(self, keywords, negative_keywords):
        candidates = set()
        for keyword in keywords:
            if keyword in self.index:
                candidates.update(self.index[keyword])
        for neg_keyword in negative_keywords:
            if neg_keyword in self.index:
                candidates.difference_update(self.index[neg_keyword])
        return candidates


In [4]:
#POWER Algorithm (TKQN)**

def POWER(quadtree, inverted_index, query_coords, keywords, negative_keywords, k):
    # Spatial filtering (kNN using quadtree)
    x, y = query_coords
    radius = 0.1  # Initial search radius (adjust as needed)
    while True:
        spatial_candidates = quadtree.query_range(x - radius, y - radius, x + radius, y + radius)
        if len(spatial_candidates) >= k:
            break
        radius *= 2  # Expand search radius

    #Keyword filtering
    keyword_candidates = inverted_index.filter(keywords, negative_keywords)

    #Intersection and ranking
    final_candidates = [point for point in spatial_candidates if point.id in keyword_candidates]
    final_candidates.sort(key=lambda p: (p.x - x) ** 2 + (p.y - y) ** 2)  # Sort by distance

    # Return top-k results
    return final_candidates[:k]


In [5]:
#Batch Processing Module
def batch_POWER(quadtree, inverted_index, queries, k):
    results = []
    for query in queries:
        query_coords, keywords, negative_keywords = query
        result = POWER(quadtree, inverted_index, query_coords, keywords, negative_keywords, k)
        results.append(result)
    return results



In [6]:
import os
import glob

def parse_line(line): 
    parts = line.strip().split()
    
    # Extract data
    object_id = int(parts[0])
    latitude = float(parts[1])
    longitude = float(parts[2])
    num_keywords = int(parts[3])
    
    keywords = []
    for i in range(num_keywords):
        keyword = parts[4 + i * 2]
        # weight = float(parts[5 + i * 2])
        keywords.append(keyword)

    return {
        "id": object_id,
        "x": latitude,
        "y": longitude,
        "keywords": keywords
    }

def parse_files(folder_path):
    data_points = []
    # Use glob to match all txt files
    file_list = glob.glob(os.path.join(folder_path, '*.txt'))
    
    for file_path in file_list:
        with open(file_path, 'r') as file:
            for line in file:
                # Parse each line and append to the data_points list
                data_point = parse_line(line)
                data_points.append(data_point)

    return data_points

folder_path = 'data' 
data_points = parse_files(folder_path)


#print(data_points[:5])
print(f"Total number of data points: {len(data_points)}")

Total number of data points: 10000000


In [9]:
x_min, y_min = -180, -90  # Minimum longitude and latitude
x_max, y_max = 180, 90    # Maximum longitude and latitude

# Initialize the quadtree
quadtree = QuadTreeNode(x_min, y_min, x_max, y_max, capacity=4, max_depth=15,depth=0)

# Insert points
for point in data_points:
    point_obj = Point(point["id"], point["x"], point["y"], point["keywords"])
    quadtree.insert(point_obj)

# Initialize the inverted index
inverted_index = InvertedIndex()




for point in data_points:
    # Create a Point object
    point_obj = Point(point["id"], point["x"], point["y"], point["keywords"])
    # Insert into the quadtree
    quadtree.insert(point_obj)
    # Insert into the inverted index
    inverted_index.insert(point_obj)


print("Quadtree and Inverted Index initialized successfully!")


Quadtree and Inverted Index initialized successfully!


In [12]:
import time


queries = [
    ((37.7749, -122.4194), ["pizza", "Italian"], ["expensive"]),  
    ((34.0522, -118.2437), ["burger", "cheese"], ["spicy"]),   
]

#Original POWER (one-at-a-time)
start_time = time.time()
for query in queries:
    res= POWER(quadtree, inverted_index, *query, k=5)
    #print("Query Results:", res)
    print("Query Results:", [point.id for point in res])
original_latency = time.time() - start_time

# Batched POWER
start_time = time.time()
res= batch_POWER(quadtree, inverted_index, queries, k=5)
#print("Query Results:", [point.id for point in res])
batched_latency = time.time() - start_time

# Compare results
print(f"Original POWER Latency: {original_latency:.4f} seconds")
print(f"Batched POWER Latency: {batched_latency:.4f} seconds")



Query Results: [6235887, 6235887, 5344085, 5344085, 9588701]
Query Results: [21631, 21631, 5299264, 5299264, 3511446]
Original POWER Latency: 34.4942 seconds
Batched POWER Latency: 31.2798 seconds
